In [1]:
import pandas as pd
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess

In [2]:
def calculate_derivatives(data):
    stations = data['Station'].unique()
    derivatives = {}

    for station in stations:
        station_data = data[data['Station'] == station].copy()
        depths = station_data['Depth [m]'].values
        temperatures = station_data['Temperature [degrees_C]'].values

        forward_diff = np.gradient(temperatures, depths)
        central_diff = np.gradient(temperatures, depths, edge_order=2)

        combined_diff = np.nanmean(np.array([forward_diff, central_diff]), axis=0)
        derivatives[station] = combined_diff

    return derivatives


def smooth_with_convolution(data, window_size=20):
    kernel = np.ones(window_size) / window_size
    return np.convolve(data, kernel, mode='same')


def find_dip(smoothed_diff, depths):
    min_index = np.argmin(smoothed_diff)
    dip_start = min_index
    dip_end = min_index

    # Поиск начала спада
    while dip_start > 0 and smoothed_diff[dip_start] <= smoothed_diff[dip_start - 1]:
        dip_start -= 1

    # Поиск конца спада
    while dip_end < len(smoothed_diff) - 1 and smoothed_diff[dip_end] <= smoothed_diff[dip_end + 1]:
        dip_end += 1

    return depths[dip_start], depths[dip_end]

In [3]:
# Путь к файлу
file_path = r"Data_1.csv"
data = pd.read_csv(file_path, delimiter=';')
data['Station'] = data['Station'].astype(str)


# Основной процесс
combined_derivatives = calculate_derivatives(data)

results = []
for station, combined_diff in combined_derivatives.items():
    depths = data[data['Station'] == station]['Depth [m]'].values

    smoothed_diff = smooth_with_convolution(combined_diff)
    loess_smoothed_diff = lowess(smoothed_diff, depths, frac=0.1)[:, 1]

    dip_start, dip_end = find_dip(loess_smoothed_diff, depths)

    results.append([station, dip_start, dip_end - dip_start])

# Сохранение результатов
results_df = pd.DataFrame(results, columns=['Station', 'Start Depth', 'Thickness'])
output_file = r"output.csv"
results_df.to_csv(output_file, index=False, sep=';')

print(f"Результаты сохранены в файл: {output_file}")

Результаты сохранены в файл: output.csv
